In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# run this cell to create a home directory where all repos will be cloned

%%bash
cd /content/drive/MyDrive/
mkdir -p home

In [6]:
# run this cell to clone and install the invertible_cl repo

%%bash
cd /content/drive/MyDrive/home/
if [ ! -d "invertible_cl" ]; then
  git clone https://github.com/mishgon/invertible_cl.git
fi
cd invertible_cl
pip install -e .

invertible_cl


In [10]:
# run this cell to update (pull) the invertible_cl repo

%%bash
cd /content/drive/MyDrive/home/invertible_cl
git pull

Already up to date.


In [ ]:
# run this cell to reinstall the invertible_cl repo (use only when reqs are changed)
%%bash
cd /content/drive/MyDrive/home/invertible_cl
pip install -e .

In [1]:
# try restarting the notebook if some imports do not work

from pathlib import Path

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor

from invertible_cl.pretrain.data import CIFAR10
from invertible_cl.pretrain.models import SimCLR
from invertible_cl.eval.models.probing import OnlineProbing


drive_root = Path('/content/drive/MyDrive')
data_root = drive_root / 'data'
exp_root = drive_root / 'experiments/invertible_cl'
repo_root = drive_root / 'home/invertible_cl'

/homes/mgoncharov/miniconda3/envs/pl-bolts/lib/python3.11/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/homes/mgoncharov/miniconda3/envs/pl-bolts/lib/python3.11/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/homes/mgoncharov/miniconda3/envs/pl-bolts/lib/python3.11/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/homes/mgoncharov/miniconda3/en

In [2]:
# hyperparams

batch_size = 256
lr = 1e-2 * batch_size / 256  # change lr proportionally to batch size
num_epochs = 1000

In [13]:
datamodule = CIFAR10(
    data_dir=data_root / 'cifar10',
    batch_size=batch_size,
    num_workers=8
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
datamodule.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model = SimCLR(
    encoder_architecture='resnet18_32x32',
    lr=lr
)

callbacks = [
    OnlineProbing(
        embed_dim=model.embed_dim,
        num_classes=datamodule.num_classes
    ),
    LearningRateMonitor()
]

logger = TensorBoardLogger(
    save_dir=exp_root,
    name=f'pretrain/cifar10/simclr/'
)

trainer = pl.Trainer(
    logger=logger,
    callbacks=callbacks,
    accelerator='gpu',
    max_epochs=num_epochs,
    gradient_clip_val=1.0,
    log_every_n_steps=10
)

trainer.fit(
    model=model,
    datamodule=datamodule
)

See https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb for using tensorboard in colab